# Imports

In [14]:
# regular imports
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

%matplotlib inline

In [15]:
# display related imports
from IPython.display import display, Image, clear_output, HTML, IFrame

# Widgets
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# to save dataframe as an image
import dataframe_image as dfi

# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Forest
from sklearn.tree import DecisionTreeClassifier

In [16]:
accepted = pd.read_pickle('data/accepted_rem_na.pkl')

In [17]:
for col in accepted.columns:
    print(col)

loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
sub_grade
emp_title
emp_length
home_ownership
annual_inc
verification_status
issue_d
loan_status
pymnt_plan
url
desc
purpose
title
addr_state
dti
delinq_2yrs
earliest_cr_line
inq_last_6mths
mths_since_last_delinq
mths_since_last_record
open_acc
pub_rec
revol_bal
revol_util
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_d
last_pymnt_amnt
next_pymnt_d
last_credit_pull_d
last_fico_range_high
last_fico_range_low
collections_12_mths_ex_med
mths_since_last_major_derog
policy_code
application_type
annual_inc_joint
dti_joint
verification_status_joint
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
total_cu_tl
inq_last_12m
acc_open_past_24mths
avg_cur_b

In [18]:
X_data = accepted.drop(['funded_amnt',
                        'funded_amnt_inv',
                        'installment',
                        'int_rate', 
                        'emp_title',
                        'grade', 
                       # 'sub_grade', 
                        'loan_status', 
                        'pymnt_plan', 
                        'last_pymnt_d', 
                        'url',
                        'desc',
                        'out_prncp',
                        'out_prncp_inv',
                        'purpose',
                        'total_pymnt',
                        'total_pymnt_inv',
                        'total_rec_prncp',
                        'total_rec_int',
                        'total_rec_late_fee',
                        'recoveries',
                        'collection_recovery_fee',
                        'last_pymnt_d',
                        'last_pymnt_amnt',
                        'next_pymnt_d',
                        'last_credit_pull_d',
                        'last_fico_range_high',
                        'policy_code',
                       'hardship_flag',
                        'deferral_term',
                        'payment_plan_start_date',
                        'orig_projected_additional_accrued_interest',
                        'disbursement_method',
                        'debt_settlement_flag',
                        'debt_settlement_flag_date',
                        'settlement_status'], axis=1)

In [19]:
def shrink_it(df, ret_percent):
    ''' returns a smaller dataframe to look at based on percent'''
    if ret_percent < 1:
        return df.sample(frac = ret_percent, random_state = 22)
    return df

In [20]:
X_data_sm = shrink_it(X_data, .001)

In [21]:
X_data_sm.shape

(2261, 100)

In [22]:
y_data = X_data_sm.sub_grade

In [23]:
X_data_sm.drop('sub_grade', axis=1, inplace=True)
X_data_sm.drop('issue_d', axis=1, inplace=True)
X_data_sm.drop('title', axis=1, inplace=True)

In [24]:
# fico scores

In [26]:
features_to_encode = list(X_data_sm.select_dtypes(include = ['object', 'category']).columns) 

In [27]:
features_to_encode

['term',
 'emp_length',
 'home_ownership',
 'verification_status',
 'addr_state',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'pub_rec',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'application_type',
 'verification_status_joint',
 'acc_now_delinq',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'open_rv_12m',
 'open_rv_24m',
 'inq_fi',
 'inq_last_12m',
 'chargeoff_within_12_mths',
 'mort_acc',
 'mths_since_recent_inq',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_rev_tl_bal_gt_0',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'pub_rec_bankruptcies',
 'tax_liens',
 'sec_app_earliest_cr_line',
 'sec_app_inq_last_6mths',
 'sec_app_mort_acc',
 'sec_app_open_act_il',
 'sec_app_chargeoff_within_12_mths',
 'sec_app_collections_12_mths_ex_med',
 'issue_year',
 'issue_month',
 'fico']

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
col_trans = make_column_transformer(
                        (OneHotEncoder(),features_to_encode),
                        remainder = "passthrough"
                        )

In [28]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(
                      min_samples_leaf=1,
                      n_estimators=1055,
                      bootstrap=True,
                      oob_score=True,
                      n_jobs=-1,
                      random_state=22,
                      max_features=20)

In [29]:
X_data_sm = pd.get_dummies(X_data_sm[features_to_encode])

seed = 22  # so that the result is reproducible
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data,y_data,test_size=0.4, random_state = seed)

In [30]:
from sklearn.pipeline import make_pipeline

rfc.fit(X_data_sm, y_data)

RandomForestClassifier(max_features=20, n_estimators=1055, n_jobs=-1,
                       oob_score=True, random_state=22)

In [31]:
rfc

RandomForestClassifier(max_features=20, n_estimators=1055, n_jobs=-1,
                       oob_score=True, random_state=22)

In [32]:
rfc.base_estimator_

DecisionTreeClassifier()

In [ ]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [33]:
rfc.feature_importances_

array([0.00744827, 0.0072916 , 0.00405157, ..., 0.007974  , 0.00616553,
       0.0014213 ])

In [34]:
feature_imp = pd.Series(rfc.feature_importances_)# .sort_values(ascending=False)
feature_imp

0       0.007448
1       0.007292
2       0.004052
3       0.010139
4       0.004884
          ...   
1495    0.005167
1496    0.005956
1497    0.007974
1498    0.006166
1499    0.001421
Length: 1500, dtype: float64

In [35]:
rfc.oob_score_

0.09332153914197258

In [36]:
for i,v in enumerate(feature_imp):
    print(X_data_sm.columns[i], v)

term_ 36 months 0.007448273573532793
term_ 60 months 0.007291604405868856
emp_length_1 year 0.004051568514301146
emp_length_10+ years 0.01013944431991447
emp_length_2 years 0.0048844852606408405
emp_length_3 years 0.004851012900433083
emp_length_4 years 0.003608967825232272
emp_length_5 years 0.004297765507353035
emp_length_6 years 0.0030165423955171586
emp_length_7 years 0.0029831818146326123
emp_length_8 years 0.0032802661502602626
emp_length_9 years 0.002483837450982811
emp_length_< 1 year 0.004767018272778695
emp_length_na 0.0037559708680398683
home_ownership_ANY 4.196061538418467e-05
home_ownership_MORTGAGE 0.00928072631175989
home_ownership_NONE 0.0
home_ownership_OTHER 0.0
home_ownership_OWN 0.005185296249232524
home_ownership_RENT 0.009377632964201087
verification_status_Not Verified 0.009034501562858316
verification_status_Source Verified 0.009495573686270693
verification_status_Verified 0.008281061542276636
addr_state_AK 0.00015823775546824586
addr_state_AL 0.0010817536917023